# 미니 의도분석기 실습

## 학습목차

0. 필요 라이브러리 임포트
 - python, konlpy, pandas, Komoran
1. 사전 데이터 확인
 - 개체명 파일, 액션-개체명 파일, 액션-패턴 파일
2. 발화 형태소 분석 (Komoran 형태소 분석기 사용)
 - 액션-패턴 파일 변경하여 저장
3. 개체명 검색
 - 일치하는 개체명이 있을 시 치환 (ex. 에어컨 -> iot_device)
4. 패턴 검색
 - 일치하는 패턴 발견 시 액션 반환, 발견하지 못하면 None 반환


# 0. 필요 라이브러리 임포트

### python, konlpy

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 3s (131 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:9.3.0-1ubuntu2).
python3-d

### pandas, Komoran

In [ ]:
import pandas as pd
from konlpy.tag import Komoran

# 1. 사전 데이터 확인

### <font color=green> 구글 드라이브 연동 </font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/"
!git clone "https://github.com/kisa036/mini_nlp.git"

/content/drive/My Drive/test3
Cloning into 'mini_nlp'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 15.42 KiB | 106.00 KiB/s, done.


In [ ]:
%cd "/content/drive/My Drive/mini_nlp"
%ls

/content/drive/My Drive/test3/mini_nlp
action_ne.csv  action_pattern.csv  mini_nlp.ipynb  ne_value.csv


### <font color=green>개체명 파일 : ne_value.csv</font>

In [ ]:
ne_value = pd.read_csv('ne_value.csv', encoding='utf-8')

In [ ]:
ne_value.head()

,ne,value
0,Iot_device,에어컨
1,lc_city,서울
2,lc_city,부산
3,dt_day,오늘
4,dt_day,내일


### <font color=green>액션-개체명 파일 : action_ne.csv</font>

In [ ]:
action_ne = pd.read_csv('action_ne.csv', encoding='utf-8')

In [ ]:
action_ne.head()

,action,ne
0,tv_volume_up,NaN
1,tv_volume_down,NaN
2,sh_device_power_on,iot_device
3,sh_device_power_off,iot_device
4,wt_search_weather,dt_day|lc_city


### <font color=green> 액션-패턴 파일 : action_pattern.csv </font>

In [ ]:
action_pattern = pd.read_csv('action_pattern.csv', encoding='utf-8')

In [ ]:
action_pattern.head()

,action,pattern
0,tv_volume_up,볼륨 올려
1,tv_volume_down,볼륨 내려
2,sh_device_power_on,iot_device 켜줘
3,sh_device_power_off,iot_device 꺼줘
4,wt_search_weather,dt_day 날씨


# 2. 발화 형태소 분석

### <font color=green>Komoran tag 종류 확인</font>

In [ ]:
komoran = Komoran()
komoran.tagset

### <font color=green> Komoran 사용 예시</font>

In [ ]:
komoran.pos("KOMORAN은 한국어 형태소 분석기입니다")

[('KOMORAN', 'SL'),
 ('은', 'JX'),
 ('한국어', 'NNP'),
 ('형태소', 'NNP'),
 ('분석기', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EC')]

In [ ]:
komoran.pos("서울 오늘 내일 에어컨 날씨 보여줘")

[('서울', 'NNP'),
 ('오늘', 'NNG'),
 ('내일', 'NNG'),
 ('에어컨', 'NNP'),
 ('날씨', 'NNP'),
 ('보이', 'VV'),
 ('어', 'EC'),
 ('주', 'VX'),
 ('어', 'EC')]

### <font color=green> 액션-패턴 파일 변경하여 저장</font>

In [ ]:
pos_words = action_pattern["pattern"].apply(lambda x: komoran.pos(x))
pos_words

0                       [(볼륨, NNP), (올리, VV), (어, EC)]
1                       [(볼륨, NNP), (내리, VV), (어, EC)]
2    [(iot, SL), (_, SW), (device, SL), (켜, VV), (어...
3    [(iot, SL), (_, SW), (device, SL), (끄, VV), (어...
4            [(dt, SL), (_, SW), (day, SL), (날씨, NNP)]
5    [(dt, SL), (_, SW), (day, SL), (날씨, NNP), (알리,...
6           [(lc, SL), (_, SW), (city, SL), (날씨, NNP)]
7    [(lc, SL), (_, SW), (city, SL), (날씨, NNP), (알리...
Name: pattern, dtype: object

##### <font color=blue> **[문제] 액션-패턴 파일의 내용을 komoran pos 함수를 이용하여 형태소 분석된 내용으로 "pattern" 컬럼에 저장하세요.** </font>

In [ ]:
# 읽어 들일 파일명 : action_pattern.csv
# komoran pos 함수 활용
# 액션-패턴 정보(action_pattern) 중 "pattern" 내용에 대해 형태소 분석 내용 반영
# '연결 어미'(EC), '보조 용언'(VX) 제거, ne 명칭(ex. dt_day) 유지
# 결과 : "action_pattern_komoran.csv" 파일로 저장

# 변경 내용 (volume_up 등의 action은 본형태 유지)
## volume_up  볼륨 올려 -> volume_up  볼륨 올리 어
## wt_search_weather	dt_day 날씨 알려줘 -> wt_search_weather	dt_day 날씨 알리

action_pattern_komoran = action_pattern

def create_pos_pattern(text):
  text = komoran.pos(text)
  words = []
  patterns = []
  
  for word in text:
    if(word[1] in ["SL", "SW"]):
      words.append(word[0])
    elif(word[1] not in ["EC", "VX"]):      
      words.append(" " + word[0])

  pattern = ''.join(words).strip()
  return pattern

action_pattern_komoran["pattern"] = action_pattern_komoran["pattern"].apply(lambda x: create_pos_pattern(x))
action_pattern_komoran.to_csv('action_pattern_komoran.csv', index=False)

In [ ]:
action_pattern_komoran = pd.read_csv('action_pattern_komoran.csv', encoding='utf-8')
action_pattern_komoran

,action,pattern
0,tv_volume_up,볼륨 올 리
1,tv_volume_down,볼륨 내리
2,sh_device_power_on,iot_device 켜
3,sh_device_power_off,iot_device 끄
4,wt_search_weather,dt_day 날씨
5,wt_search_weather,dt_day 날씨 알리
6,wt_search_weather,lc_city 날씨
7,wt_search_weather,lc_city 날씨 알리


# 3. 개체명 검색


##### <font color=blue> **[문제] 분석된 형태소 중 명사에 한하여 개체명을 검색하여 치환하세요.** </font>

In [ ]:
# 읽어 들일 파일명 : ne_value.csv
# 함수명 : replace_ne()
# komoran pos 함수 활용 : 명사(NNP, NNG)에 한하여 개체명을 검색한 후 개체명 이름(ne)로 치환
# 개체명이 치환된 패턴을 반환

ne_value = pd.read_csv('ne_value.csv', encoding='utf-8')

def replace_ne(text):
  pos = komoran.pos(text)

  for word in pos:
    if(word[1] in ["NNP", "NNG"]):
      for row in ne_value.itertuples():
          if(word[0] == row.value):
            text = text.replace(row.value, row.ne)
  return text

text = replace_ne("내일 날씨 알려줘")
text

'dt_day 날씨 알려줘'

# 4. 패턴 검색

##### <font color=blue> **[문제] 입력되는 발화에 일치되는 패턴을 있는지 검색하여 결과를 출력하세요.** </font>

In [ ]:
# 읽어 들일 파일명 : action_pattern_komoran.csv, action_ne.csv
# 함수명 : detect_pattern()

# action_ne.csv 파일에 선언된 action을 모두 점검
# replace_ne() 함수 내용 참조: 개체명 치환 시 사용
# 개체명 치환 전 해당 action 에서 허용하는 NE 인지 확인 (action_ne.csv)
# komoran.pos 함수 활용
# 일치하는 패턴 발견 시 액션 반환, 발견하지 못하면 None 반환

action_ne = pd.read_csv('action_ne.csv', encoding='utf-8')

patterns = pd.read_csv('action_pattern_komoran.csv', encoding='utf-8')
pattern_action = pd.Series(patterns.action.values, index=patterns.pattern).to_dict()

def detect_pattern(text):
  result_dict = {"pos":'', "matched pattern":'None', "detected action":'None'}
  pos = komoran.pos(text)
  result_dict["pos"] = pos

  for actions in action_ne.itertuples():
    
    # 해당 action 에서 허용하는 NE인지 확인 후 개체명 치환
    for word in pos:
      if(word[1] in ["NNP", "NNG"]):
        for row in ne_value.itertuples():
          if(word[0] == row.value and not pd.isna(actions.ne) and row.ne == actions.ne):
            text = text.replace(row.value, row.ne)

    # 일치하는 패턴이 있는지 확인
    for pattern, action in pattern_action.items():
      if text == pattern:
        result_dict["matched pattern"] = pattern
        result_dict["detected action"] = action
  return result_dict

test_pattern = "오늘 날씨"
result = detect_pattern(test_pattern)

print("test pattern: ", test_pattern)
print("pos: ", result["pos"])
print("matched pattern: ", result["matched pattern"])
print("detected action: ", result["detected action"])


test pattern:  오늘 날씨
pos:  [('오늘', 'NNG'), ('날씨', 'NNP')]
matched pattern:  dt_day 날씨
detected action:  wt_search_weather
